In [30]:
1+1

2

In [31]:
%pwd

'/Users/suyash/Desktop/End to End'

In [13]:
import os
os.chdir("../")
%pwd

'/Users/suyash/Desktop/End to End'

In [60]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    max_depth: int
    random_state: int
    n_jobs: int
    target_column: str 
    Cat_columns:str
    Num_columns:str

In [61]:
from src.IPL_Predictor.constants import *
from src.IPL_Predictor.utils.common import read_yaml, create_directories
from src.IPL_Predictor.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=Path(config.root_dir),
            train_data_path=Path(config.train_data_path),
            test_data_path=Path(config.test_data_path),
            model_name=config.model_name,
            n_estimators=self.params.RandomForest.n_estimators,
            max_depth=self.params.RandomForest.max_depth,
            random_state=self.params.RandomForest.random_state,
            n_jobs=self.params.RandomForest.n_jobs,
            target_column=self.schema.TARGET_COLUMN,,
        )

        return model_trainer_config

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import joblib
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import pandas as pd
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        X_train = train_data.drop(self.config.target_column, axis=1) 
        y_train = train_data[self.config.target_column]
        X_test = test_data.drop(self.config.target_column, axis=1)
        y_test = test_data[self.config.target_column]

        rf = RandomForestClassifier(
            n_estimators=self.config.n_estimators,
            max_depth=self.config.max_depth,
            random_state=self.config.random_state,
            n_jobs=self.config.n_jobs
        )

        rf.fit(X_train, y_train)

        joblib.dump(rf, os.path.join(self.config.root_dir, self.config.model_name))
        
        return rf

In [58]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model = model_trainer.train()
except Exception as e:
    raise e

[2025-09-24 00:26:20,096: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-09-24 00:26:20,098: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-24 00:26:20,100: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-09-24 00:26:20,101: INFO: common: created directory at : artifacts]
[2025-09-24 00:26:20,101: INFO: common: created directory at : artifacts/model_trainer]


ValueError: could not convert string to float: 'Jaipur'